# MIMIC ML/DL PROJECT

In [1]:
#@title Provide Google Credentials to Colab Runtime (May Require Manually Copy/Pasting Authentication Code)
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
# Package used for interfacing w/ BigQuery from Python
from google.cloud import bigquery

#@title Enter Google Cloud/BigQuery Project ID
project_id = 'valiant-monitor-450620-c4'

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

dataset_ref = bq_client.dataset('mimiciii_clinical', project='physionet-data')
bq_client.default_dataset = dataset_ref

## Sample query to test functional BigQuery Retrieval

In [3]:
query_1 = """
SELECT
    COUNT(DISTINCT subject_id) as total_patients,
    COUNT(*) as total_procedures
FROM
    physionet-data.mimiciii_clinical.procedures_icd
WHERE
    icd9_code BETWEEN '00' AND '86';
"""

# Execute query and convert to DataFrame
df = bq_client.query(query_1).to_dataframe()

# Display the DataFrame
print(df)

# Or for Jupyter notebook, just:
df  # This will show a nice formatted table in Jupyter

   total_patients  total_procedures
0           34889            142907


,total_patients,total_procedures
0,34889,142907


# MIMIC ML/DL PROJECT

The idea of this project wil be attempting to use the MIMIC III data in combination with Machine Learning / Deep Learning methods to create a Diagnosis Assistance model to potentially automate certain diagnoses.

In [4]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
project_id = 'valiant-monitor-450620-c4'
bq_client = bigquery.Client(project=project_id)
dataset_ref = bq_client.dataset('mimiciii_clinical', project='physionet-data')
bq_client.default_dataset = dataset_ref

In [12]:
# Query for extracting subset of diagnoses to predict
query_diagnosis = """
SELECT
  icd9_code,
  COUNT(*) as diagnosis_count,
  SHORT_TITLE
FROM
  physionet-data.mimiciii_clinical.diagnoses_icd
LEFT JOIN
  physionet-data.mimiciii_clinical.d_icd_diagnoses
USING (icd9_code)
GROUP BY
  icd9_code, SHORT_TITLE
ORDER BY
  diagnosis_count DESC
LIMIT 20;
"""

diagnosis_df = bq_client.query(query_diagnosis).to_dataframe()
print("Top 20 diagnoses in MIMICIII dataset")
diagnosis_df

Top 20 diagnoses in MIMICIII dataset


,icd9_code,diagnosis_count,SHORT_TITLE
0,4019,20703,Hypertension NOS
1,4280,13111,CHF NOS
2,42731,12891,Atrial fibrillation
3,41401,12429,Crnry athrscl natve vssl
4,5849,9119,Acute kidney failure NOS
5,25000,9058,DMII wo cmp nt st uncntr
6,2724,8690,Hyperlipidemia NEC/NOS
7,51881,7497,Acute respiratry failure
8,5990,6555,Urin tract infection NOS
9,53081,6326,Esophageal reflux


Here we have retrieved the top 20 most common diagnoses in the MIMIC III dataset, ordered by highest number of diagnoses. I will select the top 5 diagnoses for further analysis to simplify the data for model digestion.

In [15]:
# Specify top 5 for targeting
target_diagnoses = list(diagnosis_df['icd9_code'].iloc[:5])
target_diagnoses_names = list(diagnosis_df['SHORT_TITLE'].iloc[:5])
print(f"Selected diagnoses for prediction (ICD9 Code): {target_diagnoses}")
print(f"Selected diagnoses for prediction (name): {target_diagnoses_names}")

Selected diagnoses for prediction (ICD9 Code): ['4019', '4280', '42731', '41401', '5849']
Selected diagnoses for prediction (name): ['Hypertension NOS', 'CHF NOS', 'Atrial fibrillation', 'Crnry athrscl natve vssl', 'Acute kidney failure NOS']


In [20]:
# Filter the patient data with lab values and vital signs
query_patient_data = """
SELECT
  p.subject_id,
  p.gender,
  p.dob,
  a.hadm_id,
  a.admittime,
  DATETIME_DIFF(a.admittime, p.dob, YEAR) as age,
  d.icd9_code,
  d.seq_num as diagnosis_seq_num
FROM
  physionet-data.mimiciii_clinical.patients p
JOIN
  physionet-data.mimiciii_clinical.admissions a
ON
  p.subject_id = a.subject_id
JOIN
  physionet-data.mimiciii_clinical.diagnoses_icd d
ON
  a.hadm_id = d.hadm_id
WHERE
  d.icd9_code IN UNNEST(@target_codes)
  AND DATETIME_DIFF(a.admittime, p.dob, YEAR) > 0
ORDER BY
  p.subject_id, a.admittime
"""

# Configure the job to accept our target diagnoses as a parameter for analysis
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("target_codes", "STRING", target_diagnoses),
    ]
)

patient_diagnoses = bq_client.query(query_patient_data, job_config=job_config).to_dataframe()

print(f"Retrieved {len(patient_diagnoses)} diagnoses records...\nShowing Sample:\n")

patient_diagnoses.head()

Retrieved 68223 diagnoses records...
Showing Sample:



,subject_id,gender,dob,hadm_id,admittime,age,icd9_code,diagnosis_seq_num
0,3,M,2025-04-11,145834,2101-10-20 19:08:00,76,5849,3
1,3,M,2025-04-11,145834,2101-10-20 19:08:00,76,4280,6
2,9,M,2108-01-26,150750,2149-11-09 13:06:00,41,4280,3
3,9,M,2108-01-26,150750,2149-11-09 13:06:00,41,5849,4
4,9,M,2108-01-26,150750,2149-11-09 13:06:00,41,4019,6


In [21]:
# Next up will be to retrieve the lab values for these patients
# to provide some potential data that might help us
query_labs = """
SELECT
  l.subject_id,
  l.hadm_id,
  l.itemid,
  d.label as lab_name,
  l.charttime,
  l.valuenum,
  l.valueuom
FROM
  physionet-data.mimiciii.labevents l
JOIN
  physionet-data.mimiciii.d_labitems d
ON
  l.itemid = d.itemid
WHERE
  l.hadm_id IN UNNEST(@hadm_ids)
  AND l.valuenum IS NOT NULL
"""

# unique hadm_ids to remove duplicate admissions
hadm_ids = list(patient_diagnoses['hadm_id'].unique())
print(f"Total unique admissions: {len(hadm_ids)}")

hadm_subset = hadm_ids[:1000]

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("hadm_ids", "INTEGER", hadm_subset),
    ]
)

lab_data = bq_client.query(query_labs, job_config=job_config).to_dataframe()
print(f"{len(lab_data)} lab record for {len(hadm_subset)} admissions analyzed")

Total unique admissions: 36890


TypeError: Object of type int64 is not JSON serializable